# Compare 2017 Child Tax Credit to repeal

Investigates why tax units would be worse off when using the 2017 CTC than a repealed CTC, as Sean Wang discovered when running my CTC -> child benefit notebook using the PUF ([notebook](https://github.com/GoFroggyRun/Notebook/blob/master/ctc_ubi_puf.ipynb)).

Findings:
* 21k CPS records have a higher after-tax income when repealing the CTC as compared to with the 2017 CTC. 85k records have the reverse, as expected.
* 60% of these records have `n24 == 0`, which should be unaffected by any CTC changes. No records with `n24 == 0` had a lower after-tax income after repealing the CTC vs. 2017 CTC.
* The unexpected records have higher after expanded income than those with expected behavior.

*This behavior is still present when removing `"_DependentCredit_before_CTC": [False]` from the 2017 reform and when removing the elimination of `_DependentCredit_Child_c` from both reforms. This indicates the issue concerns the core CTC.*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np

In [2]:
tc.__version__

'0.20.1'

### Settings

In [3]:
pd.set_option('precision', 2)

### Utilities

In [4]:
def weighted_sum(df, col):
    return (df[col] * 1.0 * df.s006).sum()

def weighted_mean(df, col):
    return weighted_sum(df, col) / df.s006.sum()

### Create reforms

CTC repeal involves eliminating the normal CTC as well as the new dependent credit for children.

A reform to return to 2017 CTC law is used to show how the distribution of CTC benefits changed with TCJA.

In [5]:
noctc_reform = {
    2018: {
        '_CTC_c': [0],
        '_DependentCredit_Child_c': [0]
    }
}

y2017_reform = {
    2018: {
        # Current: 1400.0
        "_CTC_c": [1000.0],
        # Current: [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]
        "_CTC_ps": [[75000.0, 110000.0, 55000.0, 75000.0, 75000.0]],
        # Current: 2500.0
        "_ACTC_Income_thd": [3000.0],
        # Current: 600.0
        "_DependentCredit_Child_c": [0.0],
        # Current: True
        "_DependentCredit_before_CTC": [False]
    }
}

## Generate data

In [6]:
recs = tc.Records.cps_constructor()

In [7]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [8]:
def static_calc(recs,
                ctc_treatment='keep',
                year=2018,
                cols=['s006', 'aftertax_income', 'expanded_income',
                      'n24', 'nu18', 'e18400', 'XTOT']):
    """Creates static Calculator.

    Args:
        ctc_treatment: How the Child Tax Credit is treated. Options include:
            * 'keep': No change. Default.
            * 'repeal': End entirely.
            * 'y2017': Use 2017 law.
        year: Year to advance calculations to.
        cols: Columns to extract per Calculator record. 
            Defaults to ['s006', 'expanded_income', 'aftertax_income', 'nu18',
            'n24', 'XTOT'].
        
    Returns:
        DataFrame with `cols` and percentile, decile, and quintile of 
        after-tax income.
    """
    pol = tc.Policy()
    # Enact reform based on ctc_treatment.
    # Repeal CTC unless it's kept.
    if ctc_treatment == 'y2017':
        pol.implement_reform(y2017_reform)
    elif ctc_treatment == 'repeal':
        pol.implement_reform(noctc_reform)
    # Calculate. This is needed to calculate the revenue-neutral UBI.
    calc = tc.Calculator(records=recs, policy=pol, verbose=False)
    calc.advance_to_year(year)
    calc.calc_all()
    # Create DataFrame and add identifiers.
    df = calc.dataframe(cols)
    # Add weighted sums.
    df['s006_m'] = df.s006 / 1e6
    df['expanded_income_m'] = df.s006_m * df.expanded_income
    # Add identifier.
    df['ctc_treatment'] = ctc_treatment
    # What's the column for the ID?
    df['id'] = df.index
    return df

In [9]:
scenarios = pd.concat([
    static_calc(recs, ctc_treatment='keep'),
    static_calc(recs, ctc_treatment='y2017'),
    static_calc(recs, ctc_treatment='repeal')])

## Preprocess

In [10]:
tu = scenarios.pivot_table(values='aftertax_income',
                           index='id', columns='ctc_treatment').reset_index()
tu.columns = ['id', 'afti_keep', 'afti_repeal', 'afti_y2017']

In [11]:
# Dimensions based on tax unit and baseline.
base_aftiq = scenarios.loc[scenarios.ctc_treatment == 
                           'keep'].drop('ctc_treatment', axis=1)

In [12]:
tu = pd.merge(tu, base_aftiq, on='id')

In [13]:
tu['y2017_vs_repeal'] = np.where(tu.afti_y2017 > tu.afti_repeal, 'Higher',
                                 np.where(tu.afti_y2017 < tu.afti_repeal,
                                          'Lower', 'Same'))
tu['has_n24'] = tu.n24 > 0
tu['has_e18400'] = tu.e18400 > 0
tu['n24_lt_nu18'] = tu.n24 < tu.nu18
tu['records'] = 1

In [14]:
tu_pos = tu[tu.y2017_vs_repeal == 'Higher']
tu_neg = tu[tu.y2017_vs_repeal == 'Lower']

## Analysis

In [15]:
tu.pivot_table(index='y2017_vs_repeal', values=['records', 's006_m'],
               aggfunc=sum)

,records,s006_m
y2017_vs_repeal,,
Higher,85489,33.94
Lower,20731,4.01
Same,350245,131.94


In [16]:
x = tu.pivot_table(index='y2017_vs_repeal', columns='has_n24',
                   values=['records', 's006_m', 'expanded_income_m'], aggfunc=sum)
# There's a better way to do this using slices.
x['mean_expanded_income_false'] = x.expanded_income_m[0] / x.s006_m[0]
x['mean_expanded_income_true'] = x.expanded_income_m[1] / x.s006_m[1]
x

expanded_income_m             records           s006_m         \
has_n24                     False      True     False     True   False   True   
y2017_vs_repeal                                                                 
Higher                        NaN  2.23e+06       NaN  85489.0     NaN  33.94   
Lower                    5.01e+05  2.61e+05   12685.0   8046.0    2.81   1.20   
Same                     9.23e+06  1.66e+06  295117.0  55128.0  121.22  10.72   

                mean_expanded_income_false mean_expanded_income_true  
has_n24                                                               
y2017_vs_repeal                                                       
Higher                                 NaN                  65771.06  
Lower                            178236.16                 217137.93  
Same                              76168.83                 155030.45

In [17]:
x = tu.pivot_table(index='y2017_vs_repeal', columns='has_e18400',
                   values=['records', 's006_m', 'expanded_income_m'], aggfunc=sum)
# There's a better way to do this using slices.
x['mean_expanded_income_false'] = x.expanded_income_m[0] / x.s006_m[0]
x['mean_expanded_income_true'] = x.expanded_income_m[1] / x.s006_m[1]
x

expanded_income_m           records         s006_m          \
has_e18400                  False      True   False    True  False    True   
y2017_vs_repeal                                                              
Higher                   14658.16  2.22e+06    2799   82690   0.21   33.73   
Lower                     5724.97  7.56e+05     629   20102   0.03    3.98   
Same                    273681.52  1.06e+07   21067  329178   7.89  124.05   

                mean_expanded_income_false mean_expanded_income_true  
has_e18400                                                            
y2017_vs_repeal                                                       
Higher                            70349.01                  65742.78  
Lower                            188951.18                 189882.89  
Same                              34676.58                  85625.65

In [18]:
print(('The average people per tax unit among negative tax units is {:0.2f} '
       'vs. {:1.2f} among positive tax units.').format(
    weighted_mean(tu_neg, 'XTOT'), weighted_mean(tu_pos, 'XTOT')))

The average people per tax unit among negative tax units is 3.42 vs. 3.21 among positive tax units.


### Tax units with higher aftertax_income without CTC than with 2017 CTC

In [19]:
tu_neg[tu_neg.n24 == 0].sample(2)

,id,afti_keep,afti_repeal,afti_y2017,s006,aftertax_income,expanded_income,n24,nu18,e18400,XTOT,s006_m,expanded_income_m,y2017_vs_repeal,has_n24,has_e18400,n24_lt_nu18,records
407200,407200,137916.21,137916.21,136916.21,87.28,137916.21,170901.14,0.0,0.0,3450.11,4.0,8.73e-05,14.92,Lower,False,True,False,1
4282,4282,149996.78,149996.78,149496.78,10.95,149996.78,178375.97,0.0,0.0,0.00,3.0,1.10e-05,1.95,Lower,False,False,False,1


In [20]:
tu_neg[tu_neg.n24 > 0].sample(2)

,id,afti_keep,afti_repeal,afti_y2017,s006,aftertax_income,expanded_income,n24,nu18,e18400,XTOT,s006_m,expanded_income_m,y2017_vs_repeal,has_n24,has_e18400,n24_lt_nu18,records
331263,331263,199461.78,195461.78,194961.78,39.17,199461.78,255362.31,2.0,3.0,12894.23,5.0,3.92e-05,10.0,Lower,True,True,True,1
416099,416099,111012.68,109012.68,108935.70,61.99,111012.68,140275.02,1.0,2.0,0.00,4.0,6.20e-05,8.7,Lower,True,False,True,1


### No tax units have higher aftertax_income without CTC than with 2018 CTC

In [21]:
print(('{:0.0f} tax units have higher after-tax income without CTC than '
       'with 2018 CTC').format(tu[tu.afti_repeal > tu.afti_keep].shape[0]))

0 tax units have higher after-tax income without CTC than with 2018 CTC


## Create tax records

Start with a record with unexpected result, then re-test after simplifying the record (zeroing out various inputs) until a minimal case is identified.

Without a way to convert Records to dataframes and back, or to index Records, unclear how to do this.

In [22]:
import os
data = os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')

In [23]:
test = pd.read_csv(data).iloc[[351322, 137218, 139057, 202577]]

Examine four test records: first two with `n24 == 0` and second two with `n24 > 0`.

*Split into two cells to see all columns.*

In [24]:
test.transpose().iloc[:30]

,351322,137218,139057,202577
age_head,23,49,33,45
age_spouse,0,52,31,0
e00200p,106560,54390,288404,151125
e00900p,0,0,0,0
e02100p,0,0,0,0
e00200s,0,86117,28288,0
e00900s,0,0,0,0
e02100s,0,0,0,0
a_lineno,1,1,1,1
e00600,0,6540,0,0


In [25]:
test.transpose().iloc[30:]

,351322,137218,139057,202577
housing_ben,0,0,0,0
wic_ben,0,0,0,0
XTOT,2,3,4,3
filer,1,1,1,1
FLPDYR,2012,2014,2014,2013
MARS,1,2,2,1
e01100,213,1090,0,0
e01400,0,0,0,0
e03300,0,0,0,0
e03270,0,0,0,0


### Alter records

Zero out interest.

In [26]:
test_alt = test.copy()

In [27]:
INTEREST_COLS = ['e01100', 'e01400', 'e03300', 'e03270', 'e20400', 'e32800',
                 'e19200', 'e18500', 'e03240', 'e17500', 'e18400', 'e00900',
                 'e00650', 'e00300', 'e00400', 'e01700', 'e19800', 'e20100',
                 'e03210', 'e03150', 'e02300']

In [28]:
test_alt[INTEREST_COLS] = 0

Examine records on attributes that aren't completely zero.

In [29]:
test_alt.loc[:, (test_alt != 0).any(axis=0)].transpose()

,351322,137218,139057,202577
age_head,23,49,33,45
age_spouse,0,52,31,0
e00200p,106560,54390,288404,151125
e00200s,0,86117,28288,0
a_lineno,1,1,1,1
e00600,0,6540,0,0
s006,548,46,11,37
h_seq,35570,78900,80590,20484
ffpos,1,1,1,1
fips,29,8,35,39


### Calculate altered records

In [30]:
recs_alt = tc.Records.cps_constructor(data=test_alt)

In [31]:
scenarios_alt = pd.concat([
    static_calc(recs_alt, ctc_treatment='keep'),
    static_calc(recs_alt, ctc_treatment='y2017'),
    static_calc(recs_alt, ctc_treatment='repeal')])

### Preprocess

In [32]:
tu_alt = scenarios_alt.pivot_table(values='aftertax_income',
                                   index='id', columns='ctc_treatment').reset_index()
tu_alt.columns = ['id', 'afti_keep', 'afti_repeal', 'afti_y2017']

In [33]:
# Dimensions based on tax unit and baseline.
base_aftiq_alt = scenarios_alt.loc[scenarios_alt.ctc_treatment == 
                                   'keep'].drop('ctc_treatment', axis=1)

In [34]:
tu_alt = pd.merge(tu_alt, base_aftiq_alt, on='id')

In [35]:
tu_alt['y2017_vs_repeal'] = (
    np.where(tu_alt.afti_y2017 > tu_alt.afti_repeal, 'Higher',
             np.where(tu_alt.afti_y2017 < tu_alt.afti_repeal,
                      'Lower', 'Same')))
tu_alt['has_n24'] = tu_alt.n24 > 0
tu_alt['has_e18400'] = tu_alt.e18400 > 0
tu_alt['n24_lt_nu18'] = tu_alt.n24 < tu_alt.nu18
tu_alt['records'] = 1

In [36]:
tu_pos_alt = tu_alt[tu_alt.y2017_vs_repeal == 'Higher']
tu_neg_alt = tu_alt[tu_alt.y2017_vs_repeal == 'Lower']

### Analysis

In [37]:
tu_alt.pivot_table(index='y2017_vs_repeal', values=['records', 's006_m'],
                   aggfunc=sum)

,records,s006_m
y2017_vs_repeal,,
Lower,4,169.89


In [38]:
tu_alt

,id,afti_keep,afti_repeal,afti_y2017,s006,aftertax_income,expanded_income,n24,nu18,e18400,XTOT,s006_m,expanded_income_m,y2017_vs_repeal,has_n24,has_e18400,n24_lt_nu18,records
0,0,93488.50,93488.50,92988.50,1.45e+08,93488.50,132925.31,0.0,0.0,0.0,2.0,144.70,1.92e+07,Lower,False,False,False,1
1,1,134609.47,134609.47,134109.47,1.23e+07,134609.47,183178.39,0.0,1.0,0.0,3.0,12.29,2.25e+06,Lower,False,False,True,1
2,2,279754.07,277754.07,277254.07,2.94e+06,279754.07,382512.22,1.0,2.0,0.0,4.0,2.94,1.13e+06,Lower,True,False,True,1
3,3,133009.14,131009.14,130509.14,9.97e+06,133009.14,185842.95,1.0,1.0,0.0,3.0,9.97,1.85e+06,Lower,True,False,False,1


In [39]:
tu_alt.transpose()

,0,1,2,3
id,0,1,2,3
afti_keep,9.3e+04,1.3e+05,2.8e+05,1.3e+05
afti_repeal,9.3e+04,1.3e+05,2.8e+05,1.3e+05
afti_y2017,9.3e+04,1.3e+05,2.8e+05,1.3e+05
s006,1.4e+08,1.2e+07,2.9e+06,1e+07
aftertax_income,9.3e+04,1.3e+05,2.8e+05,1.3e+05
expanded_income,1.3e+05,1.8e+05,3.8e+05,1.9e+05
n24,0,0,1,1
nu18,0,1,2,1
e18400,0,0,0,0
